In [ ]:
# Import dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import datetime as dt

In [ ]:
# Files to load
city_data_to_load = "Resources/city_data.csv"
ride_data_to_load = "Resources/ride_data.csv"

In [ ]:
# Read the city data file and store it in a pandas dataframe
city_data_df = pd.read_csv(city_data_to_load)
city_data_df

In [ ]:
# Read the ride data file and store it in a pandas dataframe
ride_data_df = pd.read_csv(ride_data_to_load)
ride_data_df

In [ ]:
# Merge the data into a single dataframe
pyber_data_df = pd.merge(ride_data_df, city_data_df, how="left", on=["city", "city"])
pyber_data_df

In [ ]:
### PART ONE ###

In [ ]:
# Get the total number of rides
total_rides = pyber_data_df.groupby(["type"]).count()["ride_id"]
total_rides

In [ ]:
# Get the total number of drivers
total_drivers = city_data_df.groupby(["type"]).sum()["driver_count"]
total_drivers

In [ ]:
# Get the total fares
total_fares = pyber_data_df.groupby(["type"]).sum()["fare"]
total_fares

In [ ]:
# Calculate the average fare per ride by city type
avg_fare_per_ride = total_fares / total_rides
avg_fare_per_ride

In [ ]:
# Calculate the average fare per driver by city type
avg_fare_per_driver = total_fares / total_drivers
avg_fare_per_driver

In [ ]:
# Create the summary dataframe
summary_df = pd.DataFrame({
    "Total Rides": total_rides,
    "Total Drivers": total_drivers,
    "Total Fares": total_fares,
    "Average Fare per Ride": avg_fare_per_ride,
    "Average Fare per Driver": avg_fare_per_driver})
summary_df

In [ ]:
# Remove the index name
summary_df.index.name = None
summary_df

In [ ]:
# Format the summary dataframe
summary_df["Total Rides"] = summary_df["Total Rides"].map("{:,}".format)

summary_df["Total Drivers"] = summary_df["Total Drivers"].map("{:,}".format)

summary_df["Total Fares"] = summary_df["Total Fares"].map("${:,.2f}".format)

summary_df["Average Fare per Ride"] = summary_df["Average Fare per Ride"].map("${:.2f}".format)

summary_df["Average Fare per Driver"] = summary_df["Average Fare per Driver"].map("${:.2f}".format)

summary_df

In [ ]:
### PART TWO ###

In [ ]:
# Format the column names of the merged dataframe
pyber_data_df = pyber_data_df.rename(columns=
                    {"city": "City",
                    "date": "Date",
                    "fare": "Fare",
                    "ride_id": "Ride Id",
                    "driver_count": "No. Drivers",
                    "type": "City Type"})
pyber_data_df

In [ ]:
# Set the index to the Date column
pyber_data_df = pyber_data_df.set_index(["Date"])
pyber_data_df

In [ ]:
# Create a new dataframe for fares
fares_df = pyber_data_df[["City Type", "Fare"]].copy()
fares_df

In [ ]:
# Set the index to the datetime data type
fares_df.index = pd.to_datetime(fares_df.index)
fares_df

In [ ]:
# Check the dataframe to make sure the index is a datetime data type
fares_df.info()

In [ ]:
# Calculate the sum of fares by city type and date and create a new dataframe
fares_by_type_df = pd.DataFrame(fares_df.groupby(["City Type", "Date"]).sum()["Fare"])
fares_by_type_df

In [ ]:
# Reset the dataframe index
fares_by_type_df = fares_by_type_df.reset_index()
fares_by_type_df

In [ ]:
# Create a pivot table dataframe
fares_pvttable_df = pd.pivot_table(fares_by_type_df, values="Fare", index="Date", columns="City Type")
fares_pvttable_df

In [ ]:
# Create a new dataframe from 1/1/19 to 4/28/19
jan_to_april_df = fares_pvttable_df.loc[: "2019-04-28"]
jan_to_april_df

In [ ]:
# Downsample into weekly bins
weekly_df = jan_to_april_df.resample("W").sum()
weekly_df

In [ ]:
# Plot the dataframe
weekly_df.plot(figsize=(40, 10), linewidth=8)
plt.style.use('fivethirtyeight')
plt.title("Total Fare by City Type", fontsize=35)
plt.xlabel(None)
plt.xticks(fontsize=26)
plt.ylabel("Fare ($USD)", fontsize=30)
plt.yticks(fontsize=26)
plt.legend(title="City Type", fontsize=25).get_title().set_fontsize(28)

# Save the figure
plt.savefig("analysis/ChallengeFig.png")

# Show the figure
plt.show()